<a href="https://colab.research.google.com/github/YunusKhan/NectarineDemo/blob/master/Nectarine_102_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
print (" \\\
Steps Taken \n \\\
1. Tried 2 approaches - Statistical  (file named 101) and Deep Learning ( file names 102_DL) \n \\\
2. Done some data clean up , kept only right wrist position, and actions 1-5  \n \\\
3. Removed column which has position 2 as it was dead weight and did not carry significance  \n \\\
 \n \\\
Linear Regression - very low accuracy of 16  \n \\\
LogisticRegression - even lower accuracy  \n \\\
Decision Tree - 99.6 accuracy  \n \\\
RandomForestRegressor - 99.4 accuracy   \n \\\
 \n \\\
Deep Learning  \n \\\
Tried a Simple Dense Model - low accuracy of 15% with 20 epochs \n \\\
Tried a simple Conv1D in Keras - varying accuracy of 25 to 65 % \n \\\
Tried a more complex Conv1d Keras with more layers , regularization, different loss function, different optimizers and activation functions, got better accuracy \n \\\
Tried multiple optimizers but most of them convereged to single values - rmsprop, adagrad, sgd etc adadelta was a little better but stuck around 35% \n \\\
\n \\\
")

 \Steps Taken 
 \1. Tried 2 approaches - Statistical  (file named 101) and Deep Learning ( file names 102_DL) 
 \2. Done some data clean up , kept only right wrist position, and actions 1-5  
 \3. Removed column which has position 2 as it was dead weight and did not carry significance  
 \ 
 \Linear Regression - very low accuracy of 16  
 \LogisticRegression - even lower accuracy  
 \Decision Tree - 99.6 accuracy  
 \RandomForestRegressor - 99.4 accuracy   
 \ 
 \Deep Learning  
 \Tried a Simple Dense Model - low accuracy of 15% with 20 epochs 
 \Tried a simple Conv1D in Keras - varying accuracy of 25 to 65 % 
 \Tried a more complex Conv1d Keras with more layers , regularization, different loss function, different optimizers and activation functions, got better accuracy 
 \Tried multiple optimizers but most of them convereged to single values - rmsprop, adagrad, sgd etc adadelta was a little better but stuck around 35% 
 \
 \


In [0]:
from __future__ import print_function

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from tensorflow.python.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.regularizers import  l1_l2
from tensorflow.python.keras.optimizers import SGD



In [99]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
lframe = []
path = "/content/drive/My Drive/FORTH_TRACE_DATASET-master/*/*.csv"
for f in glob.glob(path):
    lframe.append(pd.read_csv(f, header=None))
df = pd.concat(lframe, ignore_index=False, sort=False, axis=0)

In [0]:
df = df[ df[0] == 2]  # dropping all other positions based data other than right wrist 
df = df[ df[11] <= 5] # dropping all other activites other than 1-5 

del df[0] # removing the first column because it is only right wrist and is dead weight 

X_scaled = preprocessing.scale(df[10].values)  # normalising the data on  time stamp as it is very varied and fluctuating so keeping it between -1 & 1
# may move it to 0 to 1 or some other thing later, right or some other scaling 
df[10] = X_scaled

In [0]:
x = df.iloc[:,:-1].values
y = df.iloc[:,10:].values.ravel()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [11]:
model = Sequential()
model.add(Dense(1024, input_dim=10, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, batch_size = 1024, epochs = 25, verbose = 2, validation_data = (x_test, y_test))
dense_dl_score = model.evaluate(x_test, y_test)
print ( 'Dense DL  ... ', dense_dl_score)
#print ( 'Dense DL Accuracy ... ', dense_dl_score[1])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1024)              11264     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 1025      
Total params: 12,289
Trainable params: 12,289
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/25
473/473 - 1s - loss: 2.0147 - accuracy: 0.2579 - val_loss: 1.0951 - val_accuracy: 0.2615
Epoch 2/25
473/473 - 1s - loss: 1.2053 - accuracy: 0.2600 - val_loss: 0.6205 - val_accuracy: 0.2615
Epoch 3/25
473/473 - 1s - loss: 0.9085 - accuracy: 0.2601 - val_loss: 0.6836 - val_accuracy: 0.2615
Epoch 4/25
473/473 - 1s - loss: 0.7475 - accuracy: 0.2606 - val_loss: 0.7104 - val_accuracy: 0.2615
Epoch 5/25
473/473 - 1s - loss: 0.6642 - accuracy: 0.2607 - val_loss: 0.7106 - val_accuracy: 0.2615
Epoch 6

In [15]:
model = Sequential()
model.add(Conv1D ( filters = 128, kernel_size = 3, activation = 'relu', input_shape = (10,1)))
model.add(Conv1D ( filters = 128, kernel_size = 3, activation = 'relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.compile(loss='mae', optimizer='adam', metrics=['accuracy'])

print(model.summary())
model.fit(x_train, y_train, batch_size = 1024, epochs = 10, verbose = 2, validation_data = (x_test, y_test))
dense_dl_score = model.evaluate(x_test, y_test)
print ( 'Dense DL  ... ', dense_dl_score[0])
print ( 'Dense DL Accuracy ... ', dense_dl_score[1])

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_8 (Conv1D)            (None, 8, 128)            512       
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 6, 128)            49280     
_________________________________________________________________
dropout_8 (Dropout)          (None, 6, 128)            0         
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 3, 128)            0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 384)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               49280     
Total params: 99,072
Trainable params: 99,072
Non-trainable params: 0
__________________________________________________

In [0]:
x = df.iloc[:,:-1].values
y = df.iloc[:,10:].values.ravel()
x = x.reshape((x.shape[0], x.shape[1], 1))
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [123]:
model = Sequential()
model.add(Conv1D(filters=256, kernel_size=3,padding='valid', activation='relu',input_shape=(10,1), strides = 1,  kernel_regularizer=l1_l2(l1=0.1, l2=0.1)))
model.add(MaxPooling1D(2,data_format='channels_last'))
model.add(Dropout(0.6))
model.add(BatchNormalization())
model.add(Conv1D(filters=128, kernel_size=3 , padding='valid', activation='relu',kernel_regularizer = l1_l2(l1=0.1, l2=0.1), strides=1))
model.add(Dropout(0.6))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation = 'softmax'))
model.compile(loss='squared_hinge', optimizer='adadelta', metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, batch_size=1024, epochs=15, verbose=2, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save("nectarine_dl1.h5")
loss,acc = model.evaluate(x_test,  y_test, verbose=2)
model.save_weights('./checkpoints/my_checkpoint')
print(loss, acc)

Model: "sequential_51"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_100 (Conv1D)          (None, 8, 256)            1024      
_________________________________________________________________
max_pooling1d_48 (MaxPooling (None, 4, 256)            0         
_________________________________________________________________
dropout_133 (Dropout)        (None, 4, 256)            0         
_________________________________________________________________
batch_normalization_89 (Batc (None, 4, 256)            1024      
_________________________________________________________________
conv1d_101 (Conv1D)          (None, 2, 128)            98432     
_________________________________________________________________
dropout_134 (Dropout)        (None, 2, 128)            0         
_________________________________________________________________
batch_normalization_90 (Batc (None, 2, 128)          

In [0]:
new_model = tf.keras.models.load_model('nectarine_dl1.h5')
new_model.fit(x_train, y_train, batch_size=1024, epochs=15, verbose=2, validation_data=(x_test, y_test))
score = new_model.evaluate(x_test, y_test, verbose=0)

Epoch 1/15
473/473 - 4s - loss: 124.3890 - accuracy: 0.1289 - val_loss: 121.5314 - val_accuracy: 0.2474
Epoch 2/15
473/473 - 3s - loss: 118.4079 - accuracy: 0.1302 - val_loss: 114.8928 - val_accuracy: 0.2471
Epoch 3/15
473/473 - 3s - loss: 111.2533 - accuracy: 0.1307 - val_loss: 107.2776 - val_accuracy: 0.2474
Epoch 4/15
